# ETL Pipelines

In [ ]:
import os
import sqlite3
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import firebase_admin
from openai import OpenAI
import plotly.express as px
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import matplotlib.pyplot as plt
from pycountry import countries
from firebase_admin import storage
from IPython.display import display
pd.options.mode.copy_on_write = True
from firebase_admin import credentials
from datetime import datetime,timedelta
from sklearn.preprocessing import StandardScaler

## 1. Data Ingestion

### 1.1 Extract Similar Types of Data

In [ ]:
# Import Similar Types of Data from CSV File
df1 = pd.read_csv('../data_source/mystery.csv', encoding='utf-16')
df2 = pd.read_csv('../data_source/gdp_data.csv', skiprows=4)
df3 = pd.read_csv('../data_source/electricity_access_percent.csv', skiprows=4)
df4 = pd.read_csv('../data_source/rural_population_percent.csv', skiprows=4)

### 1.2 Extract Population Data

In [ ]:
# Import Population Data from CSV File
p1 = pd.read_csv('../data_source/population_data.csv', skiprows=4)

In [ ]:
# Import Population Data from Database
conn = sqlite3.connect("../data_source/population_data.db")
p2 = pd.read_sql("SELECT * FROM population_data", conn)
conn.close()

In [ ]:
# Import Population Data from XML
with open('../data_source/population_data.xml') as file:    
    soup = BeautifulSoup(file, 'lxml')
    
data = []
for record in soup.find_all('record'):
    record_data = {}
    
    for field in record.find_all('field'):
        if field['name'] != "Item":
            name = field['name']
            value = field.text
            record_data[name] = value
            if field['name'] == "Country or Area":
                country_code = field['key']
                record_data['Country Code'] = country_code
            

    data.append(record_data)

p3 = pd.DataFrame(data)

In [ ]:
# Import Population Data from JSON
p4 = pd.read_json('../data_source/population_data.json')

### 1.3 Extract Projects Data

In [ ]:
# Import Projects Data from CSV File
proj = pd.read_csv('../data_source/projects_data.csv')

### 1.4 Extract Data from API

In [ ]:
api_pop = pd.read_csv("../data_source/api_population_data.csv")
api_gdp = pd.read_csv("../data_source/api_gdp_data.csv")
api_elec = pd.read_csv("../data_source/api_electricity_access_data.csv")
api_rural = pd.read_csv("../data_source/api_rural_population_data.csv")

In [ ]:
dfs = [api_pop, api_gdp, api_rural, api_elec]

for df in dfs:
    print(df.shape)
    display(df.head())

In [ ]:
proj = pd.read_csv('../data_source/projects_data.csv', low_memory=False)

## 2. Data Cleansing and Transformation

### 2.1 Define Functions

In [ ]:
# Function to Clean Columns
def clean_columns(df):
    columns_to_remove = ['index','Unnamed', 'Indicator Name', 'Indicator Code']
    return df.loc[:, ~df.columns.str.contains('|'.join(columns_to_remove))]

In [ ]:
# Function to Melt dataframes
def perform_melt(df, value_name):
    return pd.melt(df, id_vars=['Country Name', 'Country Code'], var_name='Year', value_name=value_name)

In [ ]:
# Function to Fill missing values
def fill_missing_values(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_columns] = df[numeric_columns].bfill(axis=1).ffill(axis=1)
    df.fillna(0, inplace=True)
    return df

In [ ]:
def fill_missing_values_elec(df):
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    df[numeric_columns] = df[numeric_columns].ffill(axis=1)
    df.fillna(0, inplace=True)
    return df

In [ ]:
# Function Convert Datetime to Year
def convert_year(df):
    df['Year'] = pd.to_datetime(df['Year']).dt.year
    return df

### 2.2 Process Similar Types of Data

In [ ]:
[print(df['Country Name'].duplicated().sum()) for df in [df1, df2, df3, df4]]


In [ ]:
# Clean Columns
data_frames = [df1, df2, df3, df4]
for i, df in enumerate(data_frames):
    data_frames[i] = clean_columns(df)
df1, df2, df3, df4 = data_frames

In [ ]:
data_frames = [api_pop, api_gdp, api_elec, api_rural]

for i, df in enumerate(data_frames):
    data_frames[i]["Country Name"] = df["Country Name"].replace({"Viet Nam": "Vietnam"}, regex=True)

api_pop, api_gdp, api_elec, api_rural = data_frames

In [ ]:
api_pop.shape

In [ ]:
# List of DataFrames and their corresponding API data
dfs = [df1, df2, df3, df4]
apis = [api_pop, api_gdp, api_elec, api_rural]

# Merge each DataFrame with its corresponding API data
for i in range(len(dfs)):
    dfs[i] = dfs[i].merge(apis[i], on=['Country Name', 'Country Code'])

df1, df2, df3, df4 = dfs

In [ ]:
df1

In [ ]:
dfs = [df1, df2, df3, df4]
for i, df in enumerate(dfs):
    if i == 2:
        df = fill_missing_values_elec(df)
    df = fill_missing_values(df)
    dfs[i] = df

df1, df2, df3, df4 = dfs

In [ ]:
data_labels = ["Mystery Data", "GDP Data", "Electricity Data", "Rural Population Data"]
dfs = [df1, df2, df3, df4]

for label, df in zip(data_labels, dfs):
    print(f"{label}: {df.shape}")
    display(df.head())


In [ ]:
dataframes = [df1, df2, df3, df4]
value_vars = ["Population", "GDP", "Electricity Access Percent", "Rural Population Percent"]
melted_dfs = ["myst", "gdp", "elec", "rural"]

for df, value_var, melted_df in zip(dataframes, value_vars, melted_dfs):
    globals()[melted_df] = perform_melt(df, value_var)

In [ ]:
for df in [myst, gdp, elec, rural]:
    print(df.isnull().sum())


In [ ]:
for df_name in ["myst", "gdp", "elec", "rural"]:
    globals()[df_name] = convert_year(globals()[df_name])


In [ ]:
def line_plot(df, column_name, countries):
    filtered_df = df[df['Country Name'].isin(countries)]
    
    plt.figure(figsize=(6, 4))
    for country, data in filtered_df.groupby('Country Name'):
        plt.plot(data['Year'], data[column_name], label=country)
    
    plt.title(f"{column_name} by Country")
    plt.xlabel("Year")
    plt.ylabel(column_name)
    plt.legend()
    plt.show()

In [ ]:
negara = ['Afghanistan', 'Albania', 'Honduras']
line_plot(gdp, 'GDP', negara)

In [ ]:
dfs = [myst, gdp, elec, rural]

for label, df in zip(data_labels, dfs):
    print(f"{label}: {df.shape}")
    display(df.head())

### 2.3 Process Population Data

In [ ]:
# Preprocessing Population Dataframes
p1 = clean_columns(p1)

p2 = p2.rename(columns={'Country_Name': 'Country Name', 'Country_Code': 'Country Code', 'Indicator_Name': 'Indicator Name', 'Indicator_Code': 'Indicator Code'})
p2 = clean_columns(p2)

p3.columns = ['Country Name', 'Country Code', 'Year', 'Population']
p3.replace('', np.nan, inplace=True)
p3 = p3.astype({'Country Name': str, 'Country Code': str, 'Year': int, 'Population': float})
p3 = p3.pivot(index=['Country Name', 'Country Code'], columns='Year', values='Population').reset_index()
p3.columns.name = None

p4 = clean_columns(p4)

In [ ]:
dfs = [p1, p2, p3, p4]

for df in dfs:
    print(df.shape)

In [ ]:
#Update Dataframes
p1.update(p2[p2['Country Name'].isin(p1['Country Name'])])
p1.update(p3[p3['Country Name'].isin(p1['Country Name'])])
p1.update(p4[p4['Country Name'].isin(p1['Country Name'])])

In [ ]:
# Merge Dataframes
merged_pop = pd.merge(p1, api_pop, on=['Country Name','Country Code'])
print('Duplicate values: ', merged_pop['Country Name'].duplicated().sum())
print("Population Data:", merged_pop.shape)
display(merged_pop.head())

In [ ]:
merged_pop[merged_pop.isna().any(axis=1)]

In [ ]:
# Fill missing values
merged_pop.drop_duplicates(subset=['Country Name'], inplace=True)
fill_missing_values(merged_pop)
pop = perform_melt(merged_pop, "Population")
pop = convert_year(pop)

In [ ]:
merged_pop[merged_pop.isna().any(axis=1)]

In [ ]:
pop

### 2.4 Process Projects Data

In [ ]:
# Drop Columns
kolom = ['id', 'regionname', 'countryname', 'envassesmentcategorycode', 'status', 'project_name', 'boardapprovaldate', 'closingdate', 'lendprojectcost', 'sector1']
proj = proj[kolom]

In [ ]:
print("Total rows: ", proj.shape[0])
print("Duplicated values: ", proj.duplicated().sum())
print(proj.isna().sum() / proj.shape[0] * 100)

In [ ]:
# Fill Missing Values
proj['countryname'] = proj['countryname'].str.split(';').str[0]
proj.fillna({'envassesmentcategorycode': '-'}, inplace=True)
proj['lendprojectcost'] = pd.to_numeric(proj['lendprojectcost'].str.replace(',', ''))
proj.fillna({'lendprojectcost':0}, inplace=True)
proj.fillna({'status': proj['status'].mode()[0]}, inplace=True)

In [ ]:
# Fill Missing Values
proj['boardapprovaldate'] = pd.to_datetime(proj['boardapprovaldate'])
proj['boardapprovaldate'] = proj['boardapprovaldate'].bfill()

proj['closingdate'] = pd.to_datetime(proj['closingdate'])
proj['closingdate'] = proj['closingdate'].bfill()

In [ ]:
# Replace And Regex
proj['sector1'] = proj['sector1'].replace('!$!0', 'Other')
proj['sector1'] = proj['sector1'].replace('!.+', '', regex=True)
proj['sector1'] = proj['sector1'].replace('^(\(Historic\))', '', regex=True)
proj['sector1'] = proj['sector1'].str.extract(r'^([^!$]+)')

In [ ]:
print(proj.isnull().sum())

In [ ]:
proj.head()

In [ ]:
project_group = proj[['countryname', 'boardapprovaldate', 'lendprojectcost']]

## 3. Further Data Transformation

### 3.2 Check Outliers

#### a. Projects Data

In [ ]:
# Check Outlier Projects_Data
fig = px.scatter(project_group, y="lendprojectcost", hover_name='countryname')
fig.show()

#### b. Population & GDP Data

In [ ]:
gdp_myst = pd.merge(gdp, myst, on=['Country Name', 'Country Code', 'Year'])
gdp_myst.head()

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(gdp_myst['Population'].values.reshape(-1, 1), gdp_myst['GDP'].values.reshape(-1, 1))
inputs = np.linspace(1, 8e9, num=50).reshape(-1, 1)
predictions = model.predict(inputs)

fig = px.scatter(gdp_myst, x='Population', y='GDP', hover_name='Country Name')
fig.add_scatter(x=inputs.flatten(), y=predictions.flatten(), mode='lines', name='Linear Regression', line=dict(color='red', width=2))
fig.update_xaxes(title_text='Population')
fig.update_yaxes(title_text='GDP')

fig.show()


In [ ]:
country_not_found = []

for country_code in gdp_myst['Country Code'].drop_duplicates().sort_values():
    try: 
        alpha_3 = countries.lookup(country_code).alpha_3
    except LookupError:
        country_not_found.append(country_code)

country_name = gdp_myst[['Country Name', 'Country Code']].drop_duplicates().sort_values('Country Name')

for country_code in country_not_found:
    country_name = gdp_myst.loc[gdp_myst['Country Code'] == country_code, 'Country Name'].iloc[0]
    print(f"{country_name}", end=", ")


In [ ]:
clean_gdp_myst = gdp_myst[~gdp_myst['Country Code'].isin(country_not_found)]

In [ ]:
model.fit(clean_gdp_myst['Population'].values.reshape(-1, 1), clean_gdp_myst['GDP'].values.reshape(-1, 1))

inputs = np.linspace(1, 8e9, num=50).reshape(-1, 1)
predictions = model.predict(inputs)

fig = px.scatter(clean_gdp_myst, x='Population', y='GDP', hover_name='Country Name')
fig.add_scatter(x=inputs.flatten(), y=predictions.flatten(), mode='lines', name='Linear Regression', line=dict(color='red', width=2))
fig.update_xaxes(title_text='Population')
fig.update_yaxes(title_text='GDP')

fig.show()

### 3.3 Dummy Variables

In [ ]:
status = proj.copy()
print(status['status'].sort_values().unique())

In [ ]:
dummies = pd.get_dummies(status['status'])
dummies.head()

In [ ]:
status['year'] = status['boardapprovaldate'].dt.year
year = status[['year', 'lendprojectcost']]
years_project = pd.concat([year, dummies], axis=1)
years_project

In [ ]:
country_name = status[['countryname']]
country_project = pd.concat([country_name, dummies], axis=1)
country_project

### 3.4 Feature Engineering

In [ ]:
project_group['boardapprovaldate'] = pd.to_datetime(project_group['boardapprovaldate']).dt.year
project_group = project_group[project_group['boardapprovaldate'] >= 1960].sort_values('boardapprovaldate')
df_project = project_group.groupby(['countryname', 'boardapprovaldate'])['lendprojectcost'].sum().reset_index()
print("total rows : ", df_project.shape[0])
df_project

In [ ]:
import country_converter as coco
cc = coco.CountryConverter()
match_these = pd.Series(df_project['countryname'].tolist())
short_name = cc.pandas_convert(series=match_these, to='name_short', not_found=None)
df_project['Short Name'] = short_name

In [ ]:
df_project

In [ ]:
proj_pop = df_project.merge(
    merged_pop,
    how='left',
    left_on='Short Name',
    right_on='Country Name'
)[[
    'Country Name', 'Country Code', 'boardapprovaldate', 'lendprojectcost'
]]
proj_pop.dropna(subset=['Country Name', 'Country Code'], inplace=True)
proj_pop.rename(columns={'boardapprovaldate':'year', 'lendprojectcost': 'projectcost'}, inplace=True)
proj_pop.reset_index(drop=True)

In [ ]:
gdp_pop = gdp.merge(pop, on=['Country Name', 'Country Code', 'Year'])
gdp_pop['gdppercapita'] = gdp_pop['GDP'] / gdp_pop['Population']
rural_elec = rural.merge(elec, on=['Country Name', 'Country Code', 'Year'])
proj_pop.columns = ['countryname', 'countrycode','year', 'projectcost']
gdp_pop.columns = ['countryname', 'countrycode', 'year', 'gdp', 'population', 'gdppercapita']
rural_elec.columns = ['countryname', 'countrycode', 'year', 'ruralpopulationpercent', 'electricityaccesspercent']
print(gdp_pop.shape)
print(rural_elec.shape)

In [ ]:
df_merge = gdp_pop.merge(rural_elec, on=['countryname', 'countrycode', 'year'])
df_country = df_merge.merge(proj_pop, on=['countryname', 'countrycode', 'year'], how='left')

In [ ]:
df_country.fillna(0, inplace=True)
df_country.drop_duplicates(subset=['countryname', 'countrycode', 'year'], inplace=True)
df_country.reset_index(drop=True)

### 3.5 Scaling Features

In [ ]:
std_scaler = StandardScaler()
scaled_df = df_country.copy()

scaled_df['gdp_scaled'] = std_scaler.fit_transform(scaled_df[['gdp']])
scaled_df['population_scaled'] = std_scaler.fit_transform(scaled_df[['population']])
scaled_df['gdppercapita_scaled'] = std_scaler.fit_transform(scaled_df[['gdppercapita']])
scaled_df['ruralpopulation_scaled'] = std_scaler.fit_transform(scaled_df[['ruralpopulationpercent']])
scaled_df['electricityaccess_scaled'] = std_scaler.fit_transform(scaled_df[['electricityaccesspercent']])

clm = ['gdp_scaled', 'population_scaled', 'gdppercapita_scaled', 'ruralpopulation_scaled', 'electricityaccess_scaled']
scaled_df = scaled_df[clm]

In [ ]:
scaled_df.head()

## 4. Data Load

#### 4.1 Save to CSV

In [ ]:
df_country.to_csv('../data_final/world_data.csv', index=False)

#### 4.2 Load to Database

In [ ]:
connection = sqlite3.connect('../data_final/world_data.db')
df_country.to_sql('world_data', con=connection, if_exists='replace', index=False)

## Data Visualization

In [ ]:
load_dotenv()
my_api = os.getenv("NAGA_AI_KEY")
client = OpenAI(
    api_key= my_api,
    base_url='https://api.naga.ac/v1'
    )

def generate(prompt, model="gpt-3.5-turbo", temperature=0, max_tokens=1000):
    response = client.chat.completions.create(
        model=model,
        temperature=temperature,
        max_tokens=max_tokens,
        messages=[
            {
                "role": "user",
                "content": prompt,
            },
        ],
    )
    return response.choices[0].message.content

In [ ]:
df_country = pd.read_csv('../data_final/world_data.csv')
daftar_kolom = df_country.columns.tolist()
asean_countries = ["Indonesia", "Singapore", "Malaysia", "Thailand", "Vietnam", "Philippines", "Brunei Darussalam", "Cambodia", "Lao PDR", "Myanmar", "Timor-Leste"]
df_asean = df_country[df_country['countryname'].isin(asean_countries)]
df_2023 = df_country[df_country['year'] == 2023]
columns = ['countryname', 'countrycode','year', 'projectcost']
df_project = df_country[columns]

In [ ]:
prompt_1 = (
    f"saya memiliki dataset dengan kolom: {str(daftar_kolom)}. berikan analisis saya menarik untuk visualisasi yaitu analisis statistik, distribusi, korelasi dan tren"
)

In [ ]:
analisis = generate(prompt_1)
print(analisis)

### Analisis Statistik Deskriptif

#### Statistik Deskriptif

In [ ]:
print("\nStatistik Deskriptif tahun 2023:")
display(df_2023.describe())

#### Rata-Rata GDP Per Kapita

In [ ]:
avg_gdp_per_capita = df_asean.groupby('countryname')['gdppercapita'].mean().sort_values(ascending=False)
print("\nRata-rata GDP Per Kapita tiap Negara di ASEAN:")
print(avg_gdp_per_capita)

#### Rata-Rata Persentase Populasi Dengan Akses Listrik

In [ ]:
avg_electricity_access = df_asean.groupby('countryname')['electricityaccesspercent'].mean().sort_values(ascending=False)
print("\nPersentase Populasi dengan Akses Listrik tiap Negara di ASEAN:")
print(avg_electricity_access)

#### Rata-Rata Persentase Populasi Pedesaan

In [ ]:
avg_rural_population = df_asean.groupby('countryname')['ruralpopulationpercent'].mean().sort_values(ascending=False)
print("\nPersentase Populasi dengan Rural Population:")
print(avg_rural_population)

### Analisis Distribusi

#### hisplot distribusi data negara

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

sns.histplot(data=df_2023, x='gdp', bins=20, kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distribusi GDP')
axes[0, 0].set_xlabel('GDP')
axes[0, 0].set_ylabel('Jumlah Negara')

sns.histplot(data=df_2023, x='population', bins=20, kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Distribusi Populasi')
axes[0, 1].set_xlabel('Populasi')
axes[0, 1].set_ylabel('Jumlah Negara')

sns.histplot(data=df_2023, x='gdppercapita', bins=20, kde=True, ax=axes[0, 2])
axes[0, 2].set_title('Distribusi GDP per Kapita')
axes[0, 2].set_xlabel('GDP per Kapita')
axes[0, 2].set_ylabel('Jumlah Negara')

sns.histplot(data=df_2023, x='ruralpopulationpercent', bins=20, kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Distribusi Persentase Penduduk Pedesaan')
axes[1, 0].set_xlabel('Persentase Penduduk Pedesaan')
axes[1, 0].set_ylabel('Jumlah Negara')

sns.histplot(data=df_2023, x='electricityaccesspercent', bins=20, kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Distribusi Persentase Penduduk dengan Akses Listrik')
axes[1, 1].set_xlabel('Persentase Penduduk dengan Akses Listrik')
axes[1, 1].set_ylabel('Jumlah Negara')

fig.delaxes(axes[1, 2])
fig.suptitle('Distribusi Data Negara di Tahun 2023', fontsize=20)
plt.tight_layout()
plt.show()



#### uji normalitas 

In [ ]:
indo = df_country[df_country['countryname'] == 'Indonesia']
indo.reset_index(drop=True, inplace=True)

In [ ]:
indo

In [ ]:
from scipy import stats

def normality_test(data, column):
    column_data = data[column]
    stat, p_value = stats.shapiro(column_data)

    # Menafsirkan hasil uji
    alpha = 0.05
    if p_value > alpha:
        result = "Data kolom '{}' terdistribusi normal (tidak bisa menolak H0)".format(column)
    else:
        result = "Data kolom '{}' tidak terdistribusi normal (menolak H0)".format(column)

    return result

print(normality_test(indo, 'gdp'))
print(normality_test(indo, 'population'))
print(normality_test(indo, 'gdppercapita'))
print(normality_test(indo, 'ruralpopulationpercent'))
print(normality_test(indo, 'electricityaccesspercent'))

### Analisis Korelasi

#### Heatmap korelasi

In [ ]:
plt.figure(figsize=(10, 6))
heatmap = sns.heatmap(scaled_df.corr(), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Heatmap Korelasi')
plt.show()

#### Scatter plot GDP dan Populasi

In [ ]:

fig = px.scatter(scaled_df, y='gdp_scaled', x='population_scaled', trendline='ols', trendline_color_override='red', title='Korelasi antara GDP dan Populasi')
fig.update_layout(title_x=0.5)
fig.show()

#### Scatter plot GDP dan Populasi

In [ ]:

fig2 = px.scatter(scaled_df, y='ruralpopulation_scaled', x='electricityaccess_scaled', trendline='ols', trendline_color_override='red', title='Korelasi antara Persentase Penduduk Pedesaan dan Akses Listrik')
fig2.show()

### Analisis Time Series

#### Plot Tren GDP

In [ ]:

fig = px.line(df_asean, x='year', y='gdp', color='countryname', title='Tren GDP Negara ASEAN',
              labels={'gdp': 'GDP (USD)', 'year': 'Tahun', 'countryname': 'Negara'})
fig.update_layout(title_x=0.5)
fig.show()

#### Plot Tren Populasi

In [ ]:
fig = px.line(df_asean, x='year', y='population', color='countryname', title='Tren Populasi Negara ASEAN',
              labels={'population': 'Populasi', 'year': 'Tahun', 'countryname': 'Negara'})
fig.update_layout(title_x=0.5)
fig.show()

#### Plot Tren GDP Per Kapita

In [ ]:

fig = px.line(df_asean, x='year', y='gdppercapita', color='countryname', title='Tren GDP Per Kapita Negara ASEAN',
              labels={'gdppercapita': 'GDP Per Kapita (USD)', 'year': 'Tahun', 'countryname': 'Negara'})
fig.update_layout(title_x=0.5)
fig.show()

#### Plot Tren Persentase Populasi Dengan Akses Listrik

In [ ]:

fig = px.line(df_asean, x='year', y='electricityaccesspercent', color='countryname', title='Tren Persentase Populasi dengan Akses Listrik Negara ASEAN',
              labels={'electricityaccesspercent': 'Persentase Akses Listrik', 'year': 'Tahun', 'countryname': 'Negara'})
fig.show()

#### Plot Tren Persentase Populasi Pedesaan

In [ ]:

fig = px.line(df_asean, x='year', y='ruralpopulationpercent', color='countryname', title='Tren Persentase Populasi Pedesaan Negara ASEAN',
              labels={'ruralpopulationpercent': 'Persentase Populasi Pedesaan', 'year': 'Tahun', 'countryname': 'Negara'})
fig.show()

#### Plot Tren projectcost

In [ ]:
import plotly.graph_objs as go
df_grouped = df_project.groupby('year')['projectcost'].sum().reset_index()

# Membuat plot menggunakan Plotly
plot_data = go.Bar(x=df_grouped['year'], y=df_grouped['projectcost'])

layout = go.Layout(
    title='Total Project Cost per Year',
    xaxis=dict(title='Year'),
    yaxis=dict(title='Total Project Cost')
)

fig = go.Figure(data=[plot_data], layout=layout)

# Menampilkan plot
fig.show()